In [1]:
from util.ImgMapper import *
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
from pylab import *
from io import BytesIO
from scipy.misc import imsave, toimage
import base64

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 27 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 27 days


In [2]:
df = pd.read_csv("../features/CUB/GoogLeNet_out.csv")
df.head()

,Unnamed: 0,pool5/7x7_s1.0,pool5/7x7_s1.1,pool5/7x7_s1.2,pool5/7x7_s1.3,pool5/7x7_s1.4,pool5/7x7_s1.5,pool5/7x7_s1.6,pool5/7x7_s1.7,pool5/7x7_s1.8,...,pool5/7x7_s1.1014,pool5/7x7_s1.1015,pool5/7x7_s1.1016,pool5/7x7_s1.1017,pool5/7x7_s1.1018,pool5/7x7_s1.1019,pool5/7x7_s1.1020,pool5/7x7_s1.1021,pool5/7x7_s1.1022,pool5/7x7_s1.1023
0,../data/CUB/059.California_Gull/California_Gul...,0.000000,0.000000,0,0.000000,0.021260,0.229788,0.016264,0.013772,0.023752,...,1.703952,0.000000,0.081577,0,0.000000,0.000000,0.747728,0.000000,1.758984,0
1,../data/CUB/059.California_Gull/California_Gul...,0.000000,0.000000,0,0.827677,0.000000,0.000000,0.000000,0.000000,1.997174,...,1.023096,0.000000,1.147406,0,0.000000,0.000000,0.000000,0.000000,2.118906,0
2,../data/CUB/059.California_Gull/California_Gul...,0.103846,0.000000,0,0.009672,0.075318,0.005389,0.052282,0.230118,0.764401,...,3.262793,0.069585,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.106682,0
3,../data/CUB/059.California_Gull/California_Gul...,0.008260,0.087472,0,1.478835,0.256482,0.000000,0.000000,0.266248,0.832159,...,1.839647,0.024995,0.196685,0,0.000000,0.000000,0.103746,0.000000,1.796243,0
4,../data/CUB/059.California_Gull/California_Gul...,0.000000,0.005279,0,1.729524,0.589268,0.000000,0.000000,0.000000,0.027871,...,0.108222,0.080634,6.603347,0,3.314412,0.007142,0.202285,1.614755,0.000000,0


In [3]:
image_paths = list(df.iloc[:,0])
print len(image_paths)

6033


In [4]:
tooltip_s = []

for image_path in image_paths:
    output = BytesIO()
    #img = toimage(image_data.reshape((28,28))) # Data was a flat row of 64 "pixels".
    img = Image.open(image_path).resize((100,100))
    img.save(output, format="PNG")
    contents = "<img src='data:image/png;base64,"+base64.encodestring(output.getvalue()).decode()+"'>"
    tooltip_s.append( contents )
    output.close()

tooltip_s = np.array(tooltip_s) # need to make sure to feed it as a NumPy array, not a list

In [5]:
image_features = array(df.iloc[:,1:])
image_features.shape

(6033, 1024)

In [8]:
### Reduce dimensions and apply clustering

pca = PCA(n_components=50)
data = pca.fit_transform(array(image_features))

# Initialize to use t-SNE with 2 components (reduces data to 2 dimensions). Also note high overlap_percentage.
mapper = KeplerMapper(cluster_algorithm=cluster.DBSCAN(eps=0.2, min_samples=5), 
             reducer = manifold.TSNE(), nr_cubes=20, overlap_perc=0.8, 
             link_local=False, verbose=1)

# Fit and transform data
data = mapper.fit_transform(data)


nr_cubes = 20 

overlap_perc = 0.8

link_local = False

Clusterer = DBSCAN(algorithm='auto', eps=0.2, leaf_size=30, metric='euclidean',
    min_samples=5, p=None, random_state=None)

Scaler = MinMaxScaler(copy=True, feature_range=(0, 1))



..Reducing Dimensionality using: 
	TSNE(early_exaggeration=4.0, init='random', learning_rate=1000.0,
   metric='euclidean', n_components=2, n_iter=1000, perplexity=30.0,
   random_state=None, verbose=1)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 6033
[t-SNE] Computed conditional probabilities for sample 2000 / 6033
[t-SNE] Computed conditional probabilities for sample 3000 / 6033
[t-SNE] Computed conditional probabilities for sample 4000 / 6033
[t-SNE] Computed conditional probabilities for sample 5000 / 6033
[t-SNE] Computed conditional probabilities for sample 6000 / 6033
[t-SNE] Computed conditional probabilities for sample 6033 / 6033
[t-SNE] Mean sigma: 8.081472
[t-SNE] Error after 83 

In [12]:
data[:4,:]

array([[ 0.86844995,  0.42090868],
       [ 0.85864814,  0.42549957],
       [ 0.8546097 ,  0.43607355],
       [ 0.85326766,  0.44925794]])

In [9]:
# Create the graph
complex = mapper.map(data, dimension_index=[0,1], dimension_name="t-SNE(2) 2D")

# Create the visualizations (increased the graph_gravity for a tighter graph-look.)

# Tooltips with image data for every cluster member
mapper.visualize(complex, "results-CUB-4.html", "t-SNE on CUB", graph_charge=-50, graph_gravity=0.15, custom_tooltips=tooltip_s)
  

Mapping on data shaped (6033, 2) using dimensions [0, 1]

Creating 400 hypercubes.

created 2434 edges and 340 nodes in 0:00:07.448989.

Wrote d3.js graph to 'results-CUB-4.html'


In [21]:
node_keys = complex['nodes'].keys()

In [22]:
links_keys  = complex['links'].keys()

In [36]:
mins = []
for k in links_keys:
    mins.append(min(complex['nodes'][k]))
    
print min(mins)

0


In [46]:
complex['links'][node_keys[1]]

['8_160_0.0_[8 0]_[ 0.4  0. ]',
 '9_180_0.0_[9 0]_[ 0.45  0.  ]',
 '9_182_0.0_[9 2]_[ 0.45  0.1 ]',
 '7_142_0.0_[7 2]_[ 0.35  0.1 ]',
 '7_141_0.0_[7 1]_[ 0.35  0.05]',
 '8_162_0.0_[8 2]_[ 0.4  0.1]',
 '9_181_0.0_[9 1]_[ 0.45  0.05]',
 '7_140_0.0_[7 0]_[ 0.35  0.  ]']

In [39]:
complex['nodes'][node_keys[0]]

[342,
 1500,
 1571,
 1785,
 1793,
 1796,
 1808,
 1810,
 1813,
 2164,
 2185,
 3403,
 4478,
 4497,
 4555,
 4564,
 4568,
 4570,
 4571,
 4575,
 4576,
 4676,
 4709,
 4714,
 4986,
 4991,
 5223,
 5255,
 5522,
 5523,
 5545,
 5548,
 5805,
 5816]

In [50]:
num_edges = 0
for k in links_keys:
    num_edges += len(complex['links'][k])

In [51]:
num_edges

2434

In [45]:
len(complex['nodes'].keys())

340